In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.11:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1634625863122)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession
      .builder()
      .master("local[*]")
      .appName("Spark SQL KeepCoding Base")
      .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7becb25a


# Complex

In [3]:
//1. Pintar el schema
val df = spark
      .read
      .format("json")
      .load("Data/exercise6/complex.json")
      .cache()

    df.printSchema()

root
 |-- messages: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- data: struct (nullable = true)
 |    |    |    |-- humidity: long (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |    |    |-- sensor_id: long (nullable = true)
 |-- source: string (nullable = true)
 |-- timestamp: long (nullable = true)



df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [messages: array<struct<data:struct<humidity:bigint,temperature:bigint>,sensor_id:bigint>>, source: string ... 1 more field]


In [4]:
val flatDF = df.select($"timestamp", explode($"messages"))
      .select($"timestamp", $"col.sensor_id".as("sensor_id"), $"col.data.*")

    flatDF.printSchema()
    flatDF.show

root
 |-- timestamp: long (nullable = true)
 |-- sensor_id: long (nullable = true)
 |-- humidity: long (nullable = true)
 |-- temperature: long (nullable = true)

+----------+---------+--------+-----------+
| timestamp|sensor_id|humidity|temperature|
+----------+---------+--------+-----------+
|1599758689|        1|      80|         28|
|1599758689|        2|      80|         28|
|1599758689|        3|      80|         28|
|1599758749|        1|      92|         32|
|1599758749|        2|      82|         31|
|1599758749|        3|      70|         25|
+----------+---------+--------+-----------+



flatDF: org.apache.spark.sql.DataFrame = [timestamp: bigint, sensor_id: bigint ... 2 more fields]


# Arrays

In [5]:
//1. Pintar el schema
    val df = spark
      .read
      .format("json")
      .load("Data/exercise6/arrays.json")
      .cache()

    df.printSchema()

root
 |-- locations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- metrics: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- sensor_id: long (nullable = true)
 |-- timestamp: long (nullable = true)



df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [locations: array<string>, metrics: array<bigint> ... 2 more fields]


In [6]:
//2. Obtener el max y min de los elementos del array metrics
    df.select($"timestamp", $"sensor_id", $"locations", array_max($"metrics"), array_min($"metrics"))
      .show()

+----------+---------+-----------------+------------------+------------------+
| timestamp|sensor_id|        locations|array_max(metrics)|array_min(metrics)|
+----------+---------+-----------------+------------------+------------------+
|1599758629|        1|[ZonaA, Pasillo1]|                74|                26|
|1599758629|        2|[ZonaB, Pasillo1]|                80|                31|
|1599758629|        3|[ZonaA, Pasillo2]|                60|                34|
|1599758689|        1|[ZonaA, Pasillo1]|                80|                25|
|1599758689|        2|[ZonaB, Pasillo1]|                75|                32|
|1599758689|        3|[ZonaA, Pasillo1]|                62|                35|
+----------+---------+-----------------+------------------+------------------+



In [7]:
 //3. Obtener los datos pertenecientes a la ZonaA
    df.where(array_contains($"locations", "ZonaA"))
      .show()

+-----------------+--------+---------+----------+
|        locations| metrics|sensor_id| timestamp|
+-----------------+--------+---------+----------+
|[ZonaA, Pasillo1]|[26, 74]|        1|1599758629|
|[ZonaA, Pasillo2]|[34, 60]|        3|1599758629|
|[ZonaA, Pasillo1]|[25, 80]|        1|1599758689|
|[ZonaA, Pasillo1]|[35, 62]|        3|1599758689|
+-----------------+--------+---------+----------+



In [8]:
 /*
        4. Si sabemos que:
        * La posicion 0 de metrics es igual a temperatura
        * La posicion 1 de metrics es igual a humedad
        * La posicion 0 de locations es zona
        * La posicion 1 de locations es pasillo
     */
    df.select($"timestamp", $"sensor_id",
      element_at($"locations", 1).as("zona"),
      element_at($"locations", 2).as("pasillo"),
      element_at($"metrics", 1).as("temperatura"),
      element_at($"metrics", 2).as("humedad")
    ).show


+----------+---------+-----+--------+-----------+-------+
| timestamp|sensor_id| zona| pasillo|temperatura|humedad|
+----------+---------+-----+--------+-----------+-------+
|1599758629|        1|ZonaA|Pasillo1|         26|     74|
|1599758629|        2|ZonaB|Pasillo1|         31|     80|
|1599758629|        3|ZonaA|Pasillo2|         34|     60|
|1599758689|        1|ZonaA|Pasillo1|         25|     80|
|1599758689|        2|ZonaB|Pasillo1|         32|     75|
|1599758689|        3|ZonaA|Pasillo1|         35|     62|
+----------+---------+-----+--------+-----------+-------+

